In [9]:
import requests
import hashlib
import time
import datetime
from google.colab import userdata
import pandas as pd
import sqlite3

In [10]:
public_key=userdata.get('public_key_marvel')
private_key=userdata.get('private_key_marvel')
timestamp = str(time.time())
md5_hash= timestamp + private_key + public_key
md5_hash = hashlib.md5(md5_hash.encode("utf-8")).hexdigest()

In [57]:
# Modifique a URL para o endpoint de personagens
url = f"https://gateway.marvel.com/v1/public/series?ts={timestamp}&apikey={public_key}&hash={md5_hash}"
print(url)
# Código de tratamento dos dados
response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    print(f"Total de personagens: {data['data']['total']}")
    for char in data['data']['results']:
        print(char)

https://gateway.marvel.com/v1/public/series?ts=1747747130.6742356&apikey=9a0de28d5b5c97ca58b81276073ea689&hash=9e4f31401215d768e47cd58601162d49
Total de personagens: 15702
{'id': 6026, 'title': ' (2008)', 'description': None, 'resourceURI': 'https://gateway.marvel.com/v1/public/series/6026', 'urls': [{'type': 'detail', 'url': 'http://marvel.com/comics/series/6026/_2008?utm_campaign=apiRef&utm_source=9a0de28d5b5c97ca58b81276073ea689'}], 'startYear': 2008, 'endYear': 2008, 'rating': 'Rated a', 'type': '', 'modified': '2024-07-23T09:28:06+0000', 'thumbnail': {'path': 'http://i.annihil.us/u/prod/marvel/i/mg/b/90/4bb656aacb611', 'extension': 'jpg'}, 'creators': {'available': 34, 'collectionURI': 'https://gateway.marvel.com/v1/public/series/6026/creators', 'items': [{'resourceURI': 'https://gateway.marvel.com/v1/public/creators/1950', 'name': 'Joe Andreani', 'role': 'colorist'}, {'resourceURI': 'https://gateway.marvel.com/v1/public/creators/13215', 'name': 'Rain Beredo', 'role': 'colorist'},

# Criação do banco de dados para os characters

In [11]:
DB_NAME = 'marvel_characters_teste.db'
MAX_RETRIES = 3  # Tentativas por requisição
DELAY = 1  # Atraso entre requisições em segundos

In [ ]:
 # Função para pegar todos os personagens
def get_all_characters():
     conn = sqlite3.connect(DB_NAME)
     cursor = conn.cursor()

     offset = 0
     total = None
     inserted = 0

     while True:
         for attempt in range(MAX_RETRIES):
             try:
                 # Autenticação
                 ts = str(time.time())
                 hash_input = ts + private_key + public_key
                 md5_hash = hashlib.md5(hash_input.encode()).hexdigest()

                 params = {
                     'ts': ts,
                     'apikey': public_key,
                     'hash': md5_hash,
                     'limit': 100,
                     'offset': offset
                 }

                 response = requests.get(
                     'https://gateway.marvel.com/v1/public/characters',
                     params=params,
                     timeout=10
                 )
                 response.raise_for_status()

                 data = response.json()
                 total = data['data']['total'] if total is None else total
                 results = data['data']['results']

                 # Inserir no banco
                 for char in results:
                     cursor.execute('''
                         INSERT OR REPLACE INTO characters
                         VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                     ''', (
                         char['id'],
                         char['name'],
                         char.get('description', ''),
                         char['modified'],
                         char['comics']['available'],
                         char['comics']['collectionURI'],
                         char['events']['available'],
                         char['events']['collectionURI'],
                         char['stories']['available'],
                         char['stories']['collectionURI'],
                         char['series']['available'],
                         char['series']['collectionURI'],
                     ))

                 conn.commit()
                 inserted += len(results)
                 print(f"Progresso: {inserted}/{total} ({inserted/total:.1%})")

                 # Verificar se terminou
                 if offset + len(results) >= total:
                     print("\nTodos os personagens foram armazenados!")
                     return

                 offset += 100
                 time.sleep(DELAY)
                 break  # Sai do loop de tentativas

             except requests.exceptions.RequestException as e:
                 print(f"Erro na requisição (tentativa {attempt+1}): {str(e)}")
                 time.sleep(2 ** attempt)  # Backoff exponencial
                 continue
         else:
             print("Falha após múltiplas tentativas. Abortando...")
             break

     conn.close()

In [ ]:
def get_all_comics():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    offset = 0
    total = None
    inserted = 0
    DELAY = 1  # Atraso entre requisições

    while True:
        for attempt in range(MAX_RETRIES):
            try:
                # Autenticação
                ts = str(time.time())
                hash_input = ts + private_key + public_key
                md5_hash = hashlib.md5(hash_input.encode()).hexdigest()

                params = {
                    'ts': ts,
                    'apikey': public_key,
                    'hash': md5_hash,
                    'limit': 100,
                    'offset': offset,
                    'orderBy': 'modified'  # Melhora a paginação
                }

                response = requests.get(
                    'https://gateway.marvel.com/v1/public/comics',
                    params=params,
                    timeout=10
                )
                response.raise_for_status()

                data = response.json()
                total = data['data']['total'] if total is None else total
                results = data['data']['results']

                # Inserir quadrinhos no banco
                for comic in results:
                    # Extrair dados das coleções
                    characters_uri = comic.get('characters', {}).get('collectionURI', '')
                    events_uri = comic.get('events', {}).get('collectionURI', '')
                    creators_uri = comic.get('creators', {}).get('collectionURI', '')
                    stories_uri = comic.get('stories', {}).get('collectionURI', '')

                    cursor.execute('''
                        INSERT OR REPLACE INTO comics
                        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                    ''', (
                        comic['id'],
                        comic.get('title', 'Sem título'),
                        comic.get('pageCount', 0),
                        comic.get('modified', ''),
                        characters_uri,
                        events_uri,
                        creators_uri,
                        stories_uri
                    ))

                conn.commit()
                inserted += len(results)
                print(f"Progresso: {inserted}/{total} ({inserted/total:.1%})")

                if offset + len(results) >= total:
                    print("\nTodos os quadrinhos foram armazenados!")
                    return

                offset += 100
                time.sleep(DELAY)
                break

            except requests.exceptions.RequestException as e:
                print(f"Erro na requisição (tentativa {attempt+1}): {str(e)}")
                time.sleep(2 ** attempt)
                continue
        else:
            print("Falha após múltiplas tentativas. Abortando...")
            break

    conn.close()

In [ ]:
def get_all_events():
     conn = sqlite3.connect(DB_NAME)
     cursor = conn.cursor()

     offset = 0
     total = None
     inserted = 0

     while True:
         for attempt in range(MAX_RETRIES):
             try:
                 # Autenticação
                 ts = str(time.time())
                 hash_input = ts + private_key + public_key
                 md5_hash = hashlib.md5(hash_input.encode()).hexdigest()

                 params = {
                     'ts': ts,
                     'apikey': public_key,
                     'hash': md5_hash,
                     'limit': 10,
                     'offset': offset
                 }
                 response = requests.get(
                     'https://gateway.marvel.com/v1/public/events',
                     params=params,
                     timeout=10
                 )
                 response.raise_for_status()
                 data = response.json()
                 total = data['data']['total'] if total is None else total
                 results = data['data']['results']

                 # Inserir no banco
                 for char in results:
                     cursor.execute('''
                         INSERT OR REPLACE INTO events
                         VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                     ''', (
                         char['id'],
                         char['title'],
                         char.get('description', ''),
                         char['modified'],
                         char['start'],
                         char['end'],
                         char['comics']['available'],
                         char['comics']['collectionURI'],
                         char['characters']['available'],
                         char['characters']['collectionURI'],
                         char['stories']['available'],
                         char['stories']['collectionURI'],
                         char['creators']['available'],
                         char['creators']['collectionURI'],
                         char['series']['available'],
                         char['series']['collectionURI']
                     ))

                 conn.commit()
                 inserted += len(results)
                 print(f"Progresso: {inserted}/{total} ({inserted/total:.1%})")

                 # Verificar se terminou
                 if offset + len(results) >= total:
                     print("\nTodos os eventos foram armazenados!")
                     return

                 offset += 10
                 time.sleep(DELAY)
                 break  # Sai do loop de tentativas

             except requests.exceptions.RequestException as e:
                 print(f"Erro na requisição (tentativa {attempt+1}): {str(e)}")
                 time.sleep(2 ** attempt)  # Backoff exponencial
                 continue
         else:
             print("Falha após múltiplas tentativas. Abortando...")
             break

     conn.close()

In [ ]:
 # Função para pegar todos os personagens
def get_all_events():
     conn = sqlite3.connect(DB_NAME)
     cursor = conn.cursor()

     offset = 0
     total = None
     inserted = 0

     while True:
         for attempt in range(MAX_RETRIES):
             try:
                 # Autenticação
                 ts = str(time.time())
                 hash_input = ts + private_key + public_key
                 md5_hash = hashlib.md5(hash_input.encode()).hexdigest()

                 params = {
                     'ts': ts,
                     'apikey': public_key,
                     'hash': md5_hash,
                     'limit': 10,
                     'offset': offset
                 }
                 response = requests.get(
                     'https://gateway.marvel.com/v1/public/events',
                     params=params,
                     timeout=10
                 )
                 response.raise_for_status()
                 data = response.json()
                 total = data['data']['total'] if total is None else total
                 results = data['data']['results']

                 # Inserir no banco
                 for char in results:
                     cursor.execute('''
                         INSERT OR REPLACE INTO events
                         VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                     ''', (
                         char['id'],
                         char['title'],
                         char.get('description', ''),
                         char['modified'],
                         char['start'],
                         char['end'],
                         char['comics']['available'],
                         char['comics']['collectionURI'],
                         char['characters']['available'],
                         char['characters']['collectionURI'],
                         char['stories']['available'],
                         char['stories']['collectionURI'],
                         char['creators']['available'],
                         char['creators']['collectionURI'],
                         char['series']['available'],
                         char['series']['collectionURI']
                     ))

                 conn.commit()
                 inserted += len(results)
                 print(f"Progresso: {inserted}/{total} ({inserted/total:.1%})")

                 # Verificar se terminou
                 if offset + len(results) >= total:
                     print("\nTodos os eventos foram armazenados!")
                     return

                 offset += 10
                 time.sleep(DELAY)
                 break  # Sai do loop de tentativas

             except requests.exceptions.RequestException as e:
                 print(f"Erro na requisição (tentativa {attempt+1}): {str(e)}")
                 time.sleep(2 ** attempt)  # Backoff exponencial
                 continue
         else:
             print("Falha após múltiplas tentativas. Abortando...")
             break

     conn.close()

In [ ]:
 # Função para criar o banco
def create_database():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    # Tabela principal de personagens
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS characters (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        description TEXT,
        modified TEXT,
        comics_available INTEGER,
        comics_collectionURI TEXT,
        events_available INTEGER,
        events_collectionURI TEXT,
        stories_available INTEGER,
        stories_collectionURI TEXT,
        series_available INTEGER,
        series_collectionURI TEXT
    )''')

    # Tabela de relacionamento (corrigida)
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS characters_events (
        id_character INTEGER,
        id_event INTEGER,
        PRIMARY KEY(id_character, id_event),
        FOREIGN KEY(id_character) REFERENCES characters(id),
        FOREIGN KEY(id_event) REFERENCES events(id)
    )''')

    # Tabela Comics

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS comics (
        id INTEGER PRIMARY KEY,
        title TEXT NOT NULL,
        page_count INTEGER,
        modified TEXT,
        character_collectionURI TEXT,
        events_collectionURI TEXT,
        creator_collectionURI TEXT,
        stories_collectionURI TEXT
    )''')

    # Tabela Events
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS events (
        id INTEGER PRIMARY KEY,
        title TEXT NOT NULL,
        description TEXT,
        modified TEXT,
        start TEXT,
        end TEXT,
        comics_available INTEGER,
        comics_collectionURI TEXT,
        characters_available INTEGER,
        characters_collectionURI TEXT,
        stories_available INTEGER,
        stories_collectionURI TEXT,
        creators_available INTEGER,
        creators_collectionURI TEXT,
        series_available INTEGER,
        series_collectionURI TEXT
    )''')

    conn.commit()
    conn.close()

In [ ]:
create_database()
# get_all_comics()
# get_all_characters()
# get_all_events()

 # Verificar
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM events")
print(f"\nTotal no banco: {cursor.fetchone()[0]} eventos")
conn.close()


Total no banco: 73 eventos


# Mostrando o banco marvel_full

In [60]:
def show_table_sample():
    conn = sqlite3.connect('marvel_characters_teste.db')
    df = pd.read_sql('''SELECT * FROM characters  ORDER BY events_available DESC
    ''', conn)
    conn.close()

    print("\nAmostra dos dados:")
    print(f"Dimensões: {df.shape[0]} linhas x {df.shape[1]} colunas")
    return df

show_table_sample()


Amostra dos dados:
Dimensões: 1564 linhas x 12 colunas


,id,name,description,modified,comics_available,comics_collectionURI,events_available,events_collectionURI,stories_available,stories_collectionURI,series_available,series_collectionURI
0,1009718,Wolverine,Born with super-human senses and the power to ...,2016-05-02T12:21:44+0000,2764,https://gateway.marvel.com/v1/public/character...,44,https://gateway.marvel.com/v1/public/character...,3813,https://gateway.marvel.com/v1/public/character...,761,https://gateway.marvel.com/v1/public/character...
1,1009726,X-Men,Feared and hated by humans because they're dif...,2017-01-24T15:44:42+0000,4113,https://gateway.marvel.com/v1/public/character...,43,https://gateway.marvel.com/v1/public/character...,5820,https://gateway.marvel.com/v1/public/character...,1043,https://gateway.marvel.com/v1/public/character...
2,1009610,Spider-Man (Peter Parker),"Bitten by a radioactive spider, high school st...",2021-06-30T17:32:26+0000,4540,https://gateway.marvel.com/v1/public/character...,37,https://gateway.marvel.com/v1/public/character...,6596,https://gateway.marvel.com/v1/public/character...,1223,https://gateway.marvel.com/v1/public/character...
3,1009165,Avengers,Earth's Mightiest Heroes joined forces to take...,2020-07-21T10:29:09+0000,2298,https://gateway.marvel.com/v1/public/character...,32,https://gateway.marvel.com/v1/public/character...,3129,https://gateway.marvel.com/v1/public/character...,705,https://gateway.marvel.com/v1/public/character...
4,1009368,Iron Man,"Wounded, captured and forced to build a weapon...",2016-09-28T12:08:19+0000,2745,https://gateway.marvel.com/v1/public/character...,31,https://gateway.marvel.com/v1/public/character...,4098,https://gateway.marvel.com/v1/public/character...,678,https://gateway.marvel.com/v1/public/character...
...,...,...,...,...,...,...,...,...,...,...,...,...
1559,1017853,Wave (Wave),,2021-08-27T17:43:56+0000,18,https://gateway.marvel.com/v1/public/character...,0,https://gateway.marvel.com/v1/public/character...,18,https://gateway.marvel.com/v1/public/character...,5,https://gateway.marvel.com/v1/public/character...
1560,1017854,Luna Snow (Luna Snow),,2021-08-18T23:16:11+0000,20,https://gateway.marvel.com/v1/public/character...,0,https://gateway.marvel.com/v1/public/character...,21,https://gateway.marvel.com/v1/public/character...,9,https://gateway.marvel.com/v1/public/character...
1561,1017856,Captain Britain (Betsy Braddock),,2021-08-05T18:54:18+0000,37,https://gateway.marvel.com/v1/public/character...,0,https://gateway.marvel.com/v1/public/character...,37,https://gateway.marvel.com/v1/public/character...,4,https://gateway.marvel.com/v1/public/character...
1562,1017857,Peggy Carter (Captain Carter),,2022-05-03T11:41:04+0000,17,https://gateway.marvel.com/v1/public/character...,0,https://gateway.marvel.com/v1/public/character...,17,https://gateway.marvel.com/v1/public/character...,9,https://gateway.marvel.com/v1/public/character...


In [ ]:
def show_table_sample():
    conn = sqlite3.connect('marvel_characters_teste.db')
    df = pd.read_sql('''SELECT c.name, e.title
                        FROM characters c
                        JOIN characters_events ce ON c.id = ce.id_character
                        JOIN events e ON e.id = ce.id_event ''', conn)
    conn.close()

    print("\nAmostra dos dados:")
    print(f"Dimensões: {df.shape[0]} linhas x {df.shape[1]} colunas")
    return df

show_table_sample()


Amostra dos dados:
Dimensões: 2556 linhas x 2 colunas


,name,title
0,Abomination (Emil Blonsky),Chaos War
1,Absorbing Man,Acts of Vengeance!
2,Absorbing Man,Civil War
3,Absorbing Man,Fear Itself
4,Absorbing Man,Secret Wars
...,...,...
2551,Ironheart (Riri Williams),Secret Empire
2552,War Machine (James Rhodes),Dark Reign
2553,War Machine (James Rhodes),Initiative
2554,War Machine (James Rhodes),Secret Invasion


In [4]:

conn = sqlite3.connect('marvel_characters_teste.db')
cursor = conn.cursor()

# Executar consulta com parâmetro
cursor.execute('''
    SELECT name, comics_available,series_available,	stories_available,	events_available
    FROM characters
    ORDER BY events_available
    LIMIT 10
''')

# Obter e mostrar resultados
resultados = cursor.fetchall()

In [5]:
for name, comics_available,series_available,	stories_available,	events_available	 in resultados:
  print(f"- {name}: comics_available: {comics_available} - series_available: {series_available} - stories_available: {stories_available} - events_available: {events_available}")

- A.I.M.: comics_available: 53 - series_available: 35 - stories_available: 61 - events_available: 0
- Agent Zero: comics_available: 29 - series_available: 10 - stories_available: 31 - events_available: 0
- Artiee: comics_available: 0 - series_available: 0 - stories_available: 0 - events_available: 0
- Askew-Tronics: comics_available: 0 - series_available: 0 - stories_available: 0 - events_available: 0
- Lenny Balinger: comics_available: 0 - series_available: 0 - stories_available: 0 - events_available: 0
- Batroc the Leaper: comics_available: 30 - series_available: 15 - stories_available: 37 - events_available: 0
- Battering Ram: comics_available: 2 - series_available: 2 - stories_available: 2 - events_available: 0
- Becatron: comics_available: 0 - series_available: 0 - stories_available: 0 - events_available: 0
- Bedlam: comics_available: 8 - series_available: 6 - stories_available: 7 - events_available: 0
- Beef: comics_available: 1 - series_available: 1 - stories_available: 1 - even

#Tabelas de Relacionamentos

In [61]:
import requests
from requests.exceptions import ReadTimeout, RequestException

BASE_TIMEOUT = 10

def character_events():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    # Contador de requisições
    request_count = 0

    print("[INÍCIO] Conectado ao banco de dados")

    cursor.execute('SELECT id, events_collectionURI FROM characters WHERE events_available > 0')
    resultados = cursor.fetchall()
    print(f"[BANCO] Total de personagens encontrados: {len(resultados)}")

    for idx, (id_character, events_collection) in enumerate(resultados, 1):
        print(f"\n[PERSONAGEM {idx}/{len(resultados)}] Processando ID: {id_character}")
        ts = str(time.time())
        hash_input = ts + private_key + public_key
        md5_hash = hashlib.md5(hash_input.encode()).hexdigest()

        params = {
            'ts': ts,
            'apikey': public_key,
            'hash': md5_hash,
            'limit': 50, #mudar limit?
            'offset': 0
        }

        all_results = []
        total_events = 0

        while True:
            results = None
            for attempt in range(MAX_RETRIES):
                current_timeout = BASE_TIMEOUT * (attempt + 1)
                print(f"\n[TENTATIVA {attempt + 1}/{MAX_RETRIES}]")

                try:
                    # Incrementa o contador a cada tentativa
                    request_count += 1
                    response = requests.get(
                        events_collection,
                        params=params,
                        timeout=current_timeout
                    )
                    print(f"[RESPOSTA] Status Code: {response.status_code}")

                    response.raise_for_status()
                    data = response.json()

                    if params['offset'] == 0:
                        if data:
                          total_events = data['data']['total']

                          page_results = data['data']['results']
                          all_results.extend(page_results)

                        else:
                            total_events == 0
                    break

                except ReadTimeout as e:
                    if attempt == MAX_RETRIES - 1:
                        break
                    else:
                        wait_time = 2 ** attempt
                        time.sleep(wait_time)

                except RequestException as e:
                    print(f"[ERRO] Falha na requisição: {str(e)}")
                    continue

            if params['offset'] + 50 >= total_events:
                break

            params['offset'] += 50
            time.sleep(0.5)

        if all_results:
            print(f"[BANCO] Inserindo {len(all_results)} eventos para personagem {id_character}")
            try:
                dados = [(id_character, event['id']) for event in all_results]
                cursor.executemany('''
                    INSERT OR IGNORE INTO characters_events
                    VALUES (?, ?)
                ''', dados)
                conn.commit()
            except sqlite3.Error as e:
                print(f"[ERRO BANCO] Falha ao inserir dados: {str(e)}")
                conn.rollback()

    conn.close()
    print("[BANCO] Conexão fechada")

In [62]:
character_events()

 # Verificar
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM characters_events")
print(f"\nTotal no banco: {cursor.fetchone()[0]} linhas")
conn.close()

[INÍCIO] Conectado ao banco de dados
[BANCO] Total de personagens encontrados: 696

[PERSONAGEM 1/696] Processando ID: 1009146

[TENTATIVA 1/3]
[RESPOSTA] Status Code: 200
[BANCO] Inserindo 1 eventos para personagem 1009146

[PERSONAGEM 2/696] Processando ID: 1009148

[TENTATIVA 1/3]
[RESPOSTA] Status Code: 200
[BANCO] Inserindo 5 eventos para personagem 1009148

[PERSONAGEM 3/696] Processando ID: 1009149

[TENTATIVA 1/3]
[RESPOSTA] Status Code: 200
[BANCO] Inserindo 1 eventos para personagem 1009149

[PERSONAGEM 4/696] Processando ID: 1009151

[TENTATIVA 1/3]
[RESPOSTA] Status Code: 200
[BANCO] Inserindo 1 eventos para personagem 1009151

[PERSONAGEM 5/696] Processando ID: 1009152

[TENTATIVA 1/3]
[RESPOSTA] Status Code: 200
[BANCO] Inserindo 2 eventos para personagem 1009152

[PERSONAGEM 6/696] Processando ID: 1009153

[TENTATIVA 1/3]
[RESPOSTA] Status Code: 200
[BANCO] Inserindo 5 eventos para personagem 1009153

[PERSONAGEM 7/696] Processando ID: 1009154

[TENTATIVA 1/3]
[RESPOSTA]

KeyboardInterrupt: 